In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf

print(tf.__version__)

2.0.0


In [2]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [3]:
splits = tfds.Split.ALL.subsplit(weighted=(80, 10, 10))

splits, info = tfds.load('fashion_mnist', with_info=True, as_supervised=True, split=splits)

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes


In [4]:
class_names = ['T-shirt_top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [5]:
with open('labels.txt', 'w') as f:
  f.write('\n'.join(class_names))

In [6]:
IMG_SIZE = 28

In [7]:
def format_example(image, label):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  image = image / 255.0
  return image, label

In [8]:
BATCH_SIZE = 32

In [9]:
train_batches = train_examples.cache().shuffle(num_examples//4).batch(BATCH_SIZE).map(format_example).prefetch(1)
validation_batches = validation_examples.cache().batch(BATCH_SIZE).map(format_example).prefetch(1)
test_batches = test_examples.cache().batch(1).map(format_example)

In [10]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(16, 3, activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
validation_batches

<PrefetchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [12]:
train_batches,

(<PrefetchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>,)

In [13]:
model.fit(train_batches, 
          epochs=10,
          validation_data=validation_batches)

Epoch 1/10
1750/1750 [==============================] - 55s 31ms/step - loss: 0.4327 - accuracy: 0.8446 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
1750/1750 [==============================] - 20s 11ms/step - loss: 0.2921 - accuracy: 0.8952 - val_loss: 0.2570 - val_accuracy: 0.9033
Epoch 3/10
1750/1750 [==============================] - 20s 11ms/step - loss: 0.2446 - accuracy: 0.9101 - val_loss: 0.2421 - val_accuracy: 0.9137
Epoch 4/10
1750/1750 [==============================] - 23s 13ms/step - loss: 0.2105 - accuracy: 0.9223 - val_loss: 0.2249 - val_accuracy: 0.9184
Epoch 5/10
1750/1750 [==============================] - 26s 15ms/step - loss: 0.1820 - accuracy: 0.9321 - val_loss: 0.2160 - val_accuracy: 0.9210
Epoch 6/10
1750/1750 [==============================] - 23s 13ms/step - loss: 0.1573 - accuracy: 0.9412 - val_loss: 0.2276 - val_accuracy: 0.9199
Epoch 7/10
1750/1750 [==============================] - 22s 13ms/step - loss: 0.1361 - accuracy: 0.9491 - val_loss: 

In [15]:
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/1/assets


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/1/assets


In [16]:
#@title Select mode of optimization
mode = "Speed" #@param ["Default", "Storage", "Speed"]

if mode == 'Storage':
  optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE
elif mode == 'Speed':
  optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
else:
  optimization = tf.lite.Optimize.DEFAULT

In [17]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
converter.optimizations = [optimization]
tflite_model = converter.convert()

In [18]:
tflite_model_file = 'model.tflite'

with open(tflite_model_file, "wb") as f:
  f.write(tflite_model)
